this script builds research profile of countries after internationally funded publications from a **specific** country is removed. Here the removing standard is remove a paper that funding from the specific country flows to other countries, e.g., USA funded paper is authored by China

In [1]:
import pandas as pd
import tqdm

In [2]:
pubs_path="../../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"
cntry_fund_frac_path='../../data/NationalFunding/Data/DerivedData/Derived/cntry_fund_frac.csv'

profile_path="../../../data/nf_folder/Data/DerivedData/Derived/dep-FundExchange/pub_no_foreign_top20.csv"

In [3]:
# Parameters
pubs_path = "../data/NationalFunding/Data/simulated_data/Data/DerivedData/CleanedRawData/pub.pkl"
cntry_fund_frac_path = "../data/NationalFunding/Data/simulated_data/Data/DerivedData/Derived/cntry_fund_frac.csv"
profile_path = "../data/NationalFunding/Data/simulated_data/Data/DerivedData/Derived/dependence/pub_noforeign_fund_top20.csv"


In [4]:
pub_df=pd.read_pickle(pubs_path)
pub_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,0,2014,29,1,[Canada],"[United States, Brazil, United States, Republi...",[Canada],"[Republic of Serbia, China, Brazil, United Sta...",1,0,1,1
1,1,2017,32,1,"[Russia, Spain]","[Austria, China, China, China, United States, ...","[Russia, Spain]","[China, United States, Austria, Russia, Japan]",1,1,1,1
2,2,2012,15,1,[Turkey],Not-Funded,[Turkey],Not-Funded,0,0,0,0
3,3,2010,53,1,"[United States, Iran]",[China],"[Iran, United States]",[China],1,1,0,1
4,4,2015,22,1,[Denmark],Not-Funded,[Denmark],Not-Funded,0,0,0,0


In [5]:
#get top 20 funders
def return_topn(df,n):
    df=df[df.funder!='Not-Funded']
    topn=df.groupby(['funder'])['cnt'].sum().reset_index().sort_values(
        by='cnt',ascending=False).head(n).funder.tolist()
    return topn
n=20
cntry_fund=pd.read_csv(cntry_fund_frac_path)
topn=return_topn(cntry_fund,n)

In [6]:
def get_publications_to_remove(fund_df,cntry):
    """Returns a list of publication IDs that are funded by a specific country
    and are either internationally coauthored or authored by other countries."""
    def assign_label(authors, cntry):
        
        if len(authors)>1 or authors[0]!=cntry:
            return 1
        else:
            return 0
    
    df=fund_df[fund_df.funder_distinct==cntry].copy()
    df['label']=df.apply(lambda x: assign_label(x.author_distinct,cntry),axis=1)
    removeid=list(df[df.label==1]['id'].unique())
    
    return removeid

In [7]:
def build_country_profile(pub_df, fund_df, cntry):
    """construct the research profile of rest of countries when the internationally-funded publications are removed
    """
    
    removeid=get_publications_to_remove(fund_df, cntry)
    df=pub_df[~pub_df['id'].isin(removeid)]
    df=df[['id','dis','author_distinct']].explode('author_distinct')
    df=df.groupby(['author_distinct','dis']).size().reset_index(name='count')
    
    return df

In [8]:
filter_df=pd.DataFrame()
years=pub_df['year'].unique()

for year in tqdm.tqdm(years):
    pub_year=pub_df[pub_df.year==year] #the publication in a specific year
    fund_year=pub_year[['id','author_distinct','funder_distinct']].explode('funder_distinct')
    for cntry in topn:
        country_profile=build_country_profile(pub_year, fund_year, cntry)
        country_profile['srce_cntry']=cntry
        country_profile['year']=year
        filter_df=pd.concat([filter_df,country_profile])

  0%|                                                    | 0/10 [00:00<?, ?it/s]

 10%|████▍                                       | 1/10 [00:02<00:20,  2.24s/it]

 20%|████████▊                                   | 2/10 [00:04<00:19,  2.41s/it]

 30%|█████████████▏                              | 3/10 [00:06<00:15,  2.25s/it]

 40%|█████████████████▌                          | 4/10 [00:08<00:12,  2.06s/it]

...

 70%|██████████████████████████████▊             | 7/10 [00:18<00:08,  2.83s/it]

 80%|███████████████████████████████████▏        | 8/10 [00:20<00:05,  2.50s/it]

 90%|███████████████████████████████████████▌    | 9/10 [00:22<00:02,  2.53s/it]

100%|███████████████████████████████████████████| 10/10 [00:25<00:00,  2.70s/it]

100%|███████████████████████████████████████████| 10/10 [00:25<00:00,  2.57s/it]

In [9]:
filter_df.to_csv(profile_path, index=False)